In [1]:
import tensorflow as tf

### Data


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import os


### Model

In [3]:
x = tf.placeholder(tf.float32,[None,10000])
W = tf.Variable(tf.zeros([784,10000]))
b = tf.Variable(tf.zeros([10000]))

### Training

In [ ]:
y_= tf.placeholder(tf.float32,[None,10000])
loss = tf.losses.mean_squared_error(labels,prediction)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [ ]:
init = tf.initiallize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    batch_xs, batch_ys = 
    sess.run(train_step, feed_dict={x:batch_xs, y_:batch_ys})

### Evaluation

In [ ]:
loss = tf.losses.mean_squared_error(labels,prediction)
print(sess.run(loss,feed_dict = {x:x_test,y_:y_test}))

### build graph

In [14]:
class base_model(object):
    def  __init__(self):
        self.regularizes = []
    
    def build_graph(self,featureNum):
        self.graph = tf.Graph()
        with self.graph.as_default():
            #Inputs
            with tf.name_scope('input'):
                self.ph_data = tf.placeholder(tf.float32, (self.batch_size,featureNum),'data')
                self.ph_t_values = tf.placeholder(tf.float32,(self.batch_size,10000),'t_values')
                self.ph_dropout = tf.placeholder(tf.float32, (), 'dropout')
            #Model
            op_values = self.inference(self.ph_data,self.ph_dropout)
            self.op_loss = self.loss(op_values, self.t_values, self.regularization)
            self.op_train = self.training(self.op_loss, self.learning_rate, self.decay_steps, self.decay_rate, self.momentum)
            self.op_prediction = self.prediction(op_values)
        
            #initialize variables, i.e.. weights and biases
            self.op_init = tf.global_variables_initializer()
            
            #summaries for tensorboard and save for model parameters
            self.op_summary = tf.summary.merge_all()
            self.op_saver = tf.train.Saver(max_to_keep = 5)
        self.graph.finalize()
        
    def inference(self, data, dropout):
        #it builds the computational graph, as far as is required for running the network forward to make prediciton 
        #i.e. return the lofits given raw data
        #data  size N*M
        # N:number  of signals(samples)
        # M:number of features
        values = self._inference(data, dropout)
        return values
           
    def loss(self, values, t_values, regularizarion):
        # adds to the inference model the layers required to generate loss
        with tf.name_scope('loss'):
            with tf.name_scope('mse'):
                mse = tf.square(values - t_values)
                mse = tf.reduce_mean(mse)
            with tf.name_scope('reularizarion'):
                regularization *= tf.add_n(self.regularizers)
            loss = cross_entropy + regularization
            
        #summaries for tensorboard
        tf.summary.scalar('loss/mse',mse)
        tf.summary.scalar('loss/regularization',regularization)
        tf.summary.scalar('loss/total',loss)
        with tf.name_scope('averages'):
            #calculate the average loss up to now
            pass
        
        return loss
        
    def training(self,loss, learning_rate, decay_steps, decay_rate = 0.95, momentum = 0.9):
        with tf.name_scope('training'):
            #learning rate
            global_step = tf.Variable(0, name = 'global_step',trainable = False)
            if decay_rate != 1:
                learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase = True)
            tf.summary.scalar('learning_rate', learning_rate)
            #optimizer
            if momentum == 0:
                optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            else :
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
            grads = optimizer.compute_gradients(loss)
            op_gradients = optimizer.apply_gradients(grads, global_step = global_step)
            #Histograms
            for grad ,var in grads:
                if grad is None:
                    print('warning:{} has no gradient'.format(var.op.name))
                else:
                    tf.summary.histogram(var.op.name + '/gradients', grad)
            #The op return the learning rate
            with tf.control_dependencies([op_gradients]):
                op_train = tf.identity(learning_rate, name = 'control')
            return op_train
    
    def _conv2d(self, x, W):
        return tf.nn.conv2d(x,W,strides = [1,1,1,1], padding = 'SAME')
    
    def predict(self, data, labels = None, sess = None):
        loss = 0
        size = data.shape[0]
        predictions = np.empty(size)
        sess = self._get_session(sess)
        for begin in range(0,size,self.batch_size):
            end = begin + self.batch_size
            end = min([end,size])
            
            batch_data = np.zeros((self.batch_size, data.shape[1]))
            tmp_data = data[begin:end,:]
            #convert sparse matrices
            if type(tmp_data) is not np.ndarray():
                tmp_data = tmp.data.toarray()
            batch_data[:end - begin] = tmp_data
            feed_dict = {self.ph_data: batch_data,self.ph_dropout:1}
            
            #compute the loss
            if labels is not None:
                batch_labels = np.zeros(self.batch_size)
                batch_labels[:end-begin] = labels[begin:end]
    
    def fit(self, train_data, train_y, val_data, val_labels):
        #process time
        t_process, t_wall = time.process_time(), time.time()
        sess = tf.Session(graph = self.graph)
        
        #logging the fit information
        #shutil.rmtree(self._get_path('summaries'), ignore_errors = True)
        #writer = tf.summary.FileWriter(self._get_path('summaries'),self.graph)
        #shutil.rmtree(self._get_path('checkpoints'), ignore_errors = True)
        #os.makedirs(self._get_path('checkpoints'))
        #path = os.path.join(self._get_path('checkpoints'),'model')
        sess.run(self.op_init)
        #Training
        accuracies = []
        losses = []
        indices = collections.deque()
        num_steps = int(self.num_epochs * train_data.shape[0]/self.batch_size)
        for step in range(1, num_steps+1):
            if len(indices) < self.batch_size:
                indices.extend(np.random.permutation(train_data.shape[0]))
            idx = [indices.popleft() for i in range(self.batch_size)]
            batch_size, batch_labels = train_data[idx,i],train_labels[idx]
            if type(batch_data) is not np.ndarray:
                batch_data = batch_data.toarray() # comvert sparse matrices
            feed_dict = {self.ph_data : batch_data, self.ph_labels:batch_labels, self.ph_dropout:self.dropout}
            learning_rate, loss_average = sees.run([self.op_train,self.op_loss_average],feed_dict)
        
         #periodical evaluation of the model
            if step % self.eval_frequency == 0 or step == num_steps:
                epoch = step * self.batch_size / train_data.shape[0]
                print('step{}/{} (epoch {:.2e}, loss_average = {:.2e}'.format(learning_rate,loss,average))
                mse, loss = self.evaluate(val_data,val_labels,sess)
                accuracies.append(mse)
                print(' validation {}'.format(string))
                print(' time: {:.0f}s(wall{:.0f})s'.format(time.process_time() - t_process, time.time() - t_wall))

                #Summaries for tensorboard
                summary = tf.Summary()
                summary.ParseFromString(sess.run(self.op_summary, feed_dict))
                summary.value.add(tag = 'validation/accuracy', simple_value = accuracy)
                summary.value.add(tag='validation/loss',simple_value = loss)
                write.add_summary(summary, step)

                #save model parameters for evaluation
                self.op_saver.save(sess,path,global_step = step)
        print('validation accuracy: peak = {:.2f}, mean={:.2f}'.format(max(accuracies),np.mean(accuracies[-10:])))
        writer.close()
        sess.close()
        
        t_step = (time.time() - t_wall) / num_steps
        return accuracies, losses, t_step
            
            
    def evaluate(self, data, labels, sess = None):
        #return mse and loss
        return _,_

In [15]:
#1 layer fully convolutional network
class fc1(base_model):
    def __init__(self):
        super.__init__()
    def _inference(self,x,dropout):
        W = self._weight_variable([NFEATURES,NCLASSES])
        b = self._bias_variabl([NCLASSES])
        y = tf.matmaul(x,W) + b
        return y

In [16]:
#convolutional
class cnn2(base_model):
    #simple convolutional model
    def __init__(self,K,F):
        super().__init__()
        self.K = K  #PATCH SIZE
        self.F = F  # NUMBER OF FEATURES
    
    def _inference(self, x, dropout):
        with tf.name_scope('conv1'):
            W = self._weight_varible([self.K, self.K,1,self.F])
            b = self._bias_variable([self.F])
            x_2d = tf.reshape(x,[-1,28,28,1])
            y_2d = self._conv2d(x_2d,W)+b
            y_2d = tf.nn.relu(y_2d)
        with tf.name_scope('fc1'):
            y = tf.reshape(y_2d, [-1,NFEATURES*self.F])
            W = self._weight_variable([NFEATURES*self.F,NCLASSES])
            b = self._bias_variable([NCLASSES])
            y = tf.matmul(y,W) + b
        return y        

In [17]:
def fourier(L, algo = 'eigh', k = 1):
    #return the fourier basis, i.e. the EVD of the laplacian
    def sort(lamb, U):
        idx = lamb.argsort()
        return lamb[idx], U[:,idx]
    if algo is 'eigh':
        lamb,U = np.linalg.eigh(L.toarray())
    return lamb, U    

In [19]:
class fgcnn2(base_model):
    def __init__(self,L,F):
        super().__init__()
        self.F = F
        _, self.U = graph.fourier(L)
        
    def _inference(self, x, dropout):
        #x : NSAMPLES x NFEATURES
        with tf.name_scope('gconv1'):
            #Transform to Fourier domain
            U = tf.constant(self.U, dtype = tf.float32)
            xf = tf.matmul(x,U)
            xf = tf.expand_dims(xf,1)
            xf = tf.transpose(xf)
            #Filter
            W = self._weight_variable([NFEATURES,self.F,1])
            yf = tf.matmul(W,xf)
            yf = tf.transpose(yf)
            yf = tf.reshape(yf,[-1,NFEATURES])
            #Transform back to graph domain
            Ut = tf.transpose(U)
            y = tf.matmul(yf, Ut)
            y = tf.reshape(yf,[-1,self.F,NFEATURES])
            #Bias and non-linearity
            b = self._bias_variable([1,self.F,1])
            y+=b
            y = tf.nn.relu(y)
            
        with tf.name_scope('fc1'):
            W = self._weight_variable([self.F*NFETURES, NCLASSES])
            b = self._bias_variable([NCLASSES])
            y = tf.reshape(y, [-1,self.F*FEATURES])
            y = tf.matmul(y,W)+b
        return y

In [20]:
import time

In [25]:
def showTime(t):
    tt = time.gmtime(t/1000)
    print('{}.{} {}:{}'.format(tt.tm_mon,tt.tm_mday,tt.tm_hour,tt.tm_min))

In [37]:
showTime(1383297600000)

11.1 9:20
